# Machine Learning in Julia for Calorimeter Showers - Evaluation Exercise

The notebook is divided into 3 sections:
- **Data loading and processing**: Loading data from CSV file and conversion to appropriate format.
- **Model trainings and results**: Training and evaluating chosen model architecture (with some variations).
- **Results analysis and metrics**: Brief class distribution analysis and further metrics calculations over the best model.


## Data loading and processing

The first step is to parse the data from the CSV file into memory using the CSV package.
In order to keep the full precision of the data, we specify the types of x,y and z as Float64, otherwise with Float32 we only get an approximation to the original data. For the labels (or 'class'), I took advantage of the option `truestrings` and `falsestrings` to cast it to Bool at this step, as we are dealing with a binary classification task. ``true`` will represent _signal_ and `false` will represent _background_.

In [6]:
using CSV

# Parse data from CSV
dataset = CSV.File("dataset.csv"; types=[Float64, Float64, Float64, Bool], truestrings=["s"], falsestrings=["b"])

100000-element CSV.File:
 CSV.Row: (x = 0.58878338160622, y = 0.9778695463299387, z = 0.16442081595001978, class = false)
 CSV.Row: (x = 0.030203874012626808, y = 0.17485154297335925, z = 0.42085624829015233, class = false)
 CSV.Row: (x = 0.43158590432012234, y = 0.13594758441332477, z = 0.7539935247760404, class = false)
 CSV.Row: (x = 0.24856135748806796, y = 0.5615502817363871, z = 0.6668661845346348, class = true)
 CSV.Row: (x = 0.866113292631795, y = 0.8507238106526906, z = 0.6203192461872296, class = false)
 CSV.Row: (x = 0.6909506664428711, y = 0.08706513530885096, z = 0.36502105391029405, class = false)
 CSV.Row: (x = 0.13526441858392746, y = 0.8270012116409612, z = 0.19589886677480728, class = false)
 CSV.Row: (x = 0.9758407299380498, y = 0.9643967973466, z = 0.4987574751920051, class = false)
 CSV.Row: (x = 0.1976306469719611, y = 0.6519358453976478, z = 0.9786280059937612, class = false)
 CSV.Row: (x = 0.8833091374935147, y = 0.6096708387178466, z = 0.6505518348936886, class

By analysing the previous output, it appears that the data doesn't need normalization as it seems it's already in [0,1] range. Still, we can be sure by using the Statistics.jl package.

In [33]:
using Statistics, Printf

vectorstats(vec::Vector) = @sprintf "Max: %.3f Min: %.3f Mean: %.3f Std: %.3f" maximum(vec) minimum(vec) mean(vec) std(vec)

@show vectorstats(dataset.x);
@show vectorstats(dataset.y);
@show vectorstats(dataset.z);

vectorstats(dataset.x) = "Max: 1.000 Min: 0.000 Mean: 0.498 Std: 0.289"
vectorstats(dataset.y) = "Max: 1.000 Min: 0.000 Mean: 0.501 Std: 0.289"
vectorstats(dataset.z) = "Max: 1.000 Min: 0.000 Mean: 0.501 Std: 0.289"


As the data looks ready to be used, we need to separate the predictors from the labels. Additionally, as we'll use Flux DataLoaders, we'll take this chance to convert the data to Matrix format.

In [3]:
coords = [dataset.x'; dataset.y'; dataset.z']

3×100000 Matrix{Float64}:
 0.588783  0.0302039  0.431586  0.248561  …  0.335862  0.562414  0.683407
 0.97787   0.174852   0.135948  0.56155      0.6955    0.813789  0.140959
 0.164421  0.420856   0.753994  0.666866     0.828747  0.668709  0.185156

In [4]:
labels = dataset.class'

1×100000 adjoint(::Vector{Bool}) with eltype Bool:
 0  0  0  1  0  0  0  0  0  0  0  1  0  …  0  0  1  1  0  1  1  0  0  1  0  0

Now that we have the predictors and labels separated and in matrices, we are ready to create the dataloaders. Althought not specifically stated, I'm splitting the data into training and validation sets, so that we can validate we're not overfitting on training data. We'll still calculate the accuracy over the full dataset in the end, but this allows us to keep an eye on the progress throughtout the training run.

We split the data 85% for training and 15% for validation, and shuffle the data for training and not for validation (even though we could also shuffle it for validation, only the order is different, not the actual samples yielded).
We select batch size as 64, for we are working with tabular data and there is no concern with high memory demands.

In [5]:
using Flux

# Create dataloaders
dataloader = Flux.DataLoader((coords, labels), batchsize=64, shuffle=false) # Full dataset for final evaluation
train_dataloader = Flux.DataLoader((coords[:,begin:85_000], labels[:,begin:85_000]), batchsize=64, shuffle=true) # Data split for training
val_dataloader = Flux.DataLoader((coords[:,85_001:end], labels[:,85_001:end]), batchsize=64, shuffle=false) # Data split for validation after each epoch

235-element DataLoader(::Tuple{Matrix{Float64}, LinearAlgebra.Adjoint{Bool, Vector{Bool}}}, batchsize=64)
  with first element:
  (3×64 Matrix{Float64}, 1×64 adjoint(::Vector{Bool}) with eltype Bool,)

## Model trainings and results

For training and evaluation the following functions were created. These will work for any binary classification task.

The evaluation function takes a dataloader and a model. It runs the data through the model and compares the predictions with the labels. It calculates the accuracy, which is the ratio between correctly predicted samples and all samples in the dataloader.

The training function takes two dataloaders (one for training and another for validation) and the model to be trained. The two main concepts of this function are epochs and steps, which are represented by the two for-loops.

##### Step
The DataLoader yields a batch of data (in this case with 64 samples) that is passed as input to the model, which outputs predictions with values between 0 and 1, thanks to the sigmoid activation of the output dense layer. Again, 0 represents `background` and 1 represents `signal`. Based on those predictions, we use the loss function `binarycrossentropy` to calculate a scalar value that tells us how close those predictions are from the ground-truth labels. Finally, that loss value is used to calculate the gradients and the _Adam_ optimiser, a method of gradient descent, slighty nudges the weights into the direction of decreasing the loss. This is the essence of the learning process.

##### Epoch
One epoch is a pass over all the batches that make the dataset, on which each sample is seen once. At the end of each epoch, we calculate the accuracy over the validation dataset and the displayed loss is reset after each epoch. We also save checkpoints of the model every 50 epochs, even though the training is very quick.

The training process can be interactively monitored via logs printed with the package ProgressMeter.

In [9]:
using ProgressMeter, JLD2

"""
Calculate accuracy ``\\frac{TP+TN}{TP+TN+FP+FN}``.

Predictions are between 0 and 1, values above threshold of 0.5 are considered **signal**, otherwise are considered **background**.
Iterate through dataloader to compare predictions with ground-truths, then use reduce with hcat to combine all batches into a single BitMatrix,
where 1 is a true prediction (either TP or TN) and 0 is a false prediction (FP or FN). Given this, the accuracy is calculated with the mean.

# Arguments
- `model`: Model to evaluate.
- `dataloader::Flux.MLUtils.DataLoader`: Dataset to be evaluated.
"""
accuracy(model, dataloader::Flux.MLUtils.DataLoader) = reduce(hcat, [(model(x) .> 0.5) .== y for (x,y) in dataloader]) |> mean


"""
Train binary classification model with progress meter and checkpoints.

# Arguments
- `model`: Model to train.
- `train_dataloader::Flux.MLUtils.DataLoader`: Dataset used for training.
- `val_dataloader::Flux.MLUtils.DataLoader`: Dataset used for validation after each epoch.
- `epochs::Integer=200`: Number of epochs.
- `optimiser::Flux.Optimise.AbstractOptimiser=Adam(0.01)`: Dataset to be evaluated.
"""
function train!(
    model,
    train_dataloader::Flux.MLUtils.DataLoader,
    val_dataloader::Flux.MLUtils.DataLoader,
    epochs::Integer=200,
    optimiser::Flux.Optimise.AbstractOptimiser=Adam(0.01)
)
    p = Progress(epochs*length(train_dataloader); showspeed=true)

    optim = Flux.setup(optimiser, model)
    val_acc = NaN

    for epoch in 1:epochs
        losses = []
        for (step, (x, y)) in enumerate(train_dataloader)
            loss, grads = Flux.withgradient(model) do m
                Flux.binarycrossentropy(m(x), y)
            end
            Flux.update!(optim, model, grads[begin])

            push!(losses, loss)
            next!(p, showvalues=[(:epoch, epoch), (:step,step), (:loss,mean(losses)), (:val_acc, val_acc)])
        end

        # Calculate accuracy over validation set
        val_acc = accuracy(model, val_dataloader)
    
        # Save model checkpoints
        if epoch % 50 == 0
            jldsave("model-checkpoint-$(epoch).jld2", model_state = Flux.state(model))
        end
    end
end;

Finally, we can go over the chosen architecure and results. I chose to use the Multi-Layer Perceptron (MLP) architecture, as it's a simple and effective deep learning architecture commonly used for these types of tasks with tabular data. Instead of arbitrarily choosing an architecture, we'll start with the most basic model possible and add more layers and/or nodes at each step.

All the models have in common the input and output layers, they differ on the hidden layers. The input layer has 3 nodes that correspond to the data dimensionality (x,y and z), and the output layer has only 1 node with sigmoid activation. Alternatively, we could also have the output dense layer with 2 nodes and softmax activation, but on the specific case of binary classification, these approaches are equivalent.

I would also like to point out that we are using Float64 data, and Flux automatically converts it to Float32 unless the weights are also Float64, so I'm using the f64 function call to avoid that convertion. All of the trainings are over 200 epochs over the same data.

First of all, we have the most barebones architecture possible. No hidden layers, just the weights and biases from input to output.

> **_NOTE:_** ProgressMeter.jl is meant to work on the terminal, but here it prints a lot of text instead of refreshing, so I manually deleted all the output except the last one, for clarity sake.

In [13]:
mlp_0hidden = Dense(3 => 1, sigmoid) |> f64

train!(mlp_0hidden, train_dataloader, val_dataloader, 200)
accuracy(mlp_0hidden, dataloader)

Progress: 100%|███████████████████████████| Time: 0:00:13 (49.77 μs/it)
  epoch:    200
  step:     1329
  loss:     0.5325554185712953
  val_acc:  0.7750666666666667


0.77555

After the 13 seconds of training, we get a validation accuracy of 0.775 and overall accuracy of 0.776. Not bad for a first try, but we can add more parameters to the network to try to perform better.

On the next architecture, we add one hidden layer with 8 nodes, which increases the trainable parameters available and potencially contributing to better predictions.

In [7]:
mlp_1hidden_8 = Chain(
    Dense(3 => 8, relu),
    Dense(8 => 1, sigmoid),
) |> f64

train!(mlp_1hidden_8, train_dataloader, val_dataloader, 200)
accuracy(mlp_1hidden_8, dataloader)

Progress: 100%|███████████████████████████| Time: 0:00:18 (68.98 μs/it)
  epoch:    200
  step:     1329
  loss:     0.31322242144608164
  val_acc:  0.8360666666666666


0.83562

Indeed, the accuracy improved! We can try increasing the number of nodes in that layer even more.

In [8]:
mlp_1hidden_16 = Chain(
    Dense(3 => 16, relu),
    Dense(16 => 1, sigmoid),
) |> f64

train!(mlp_1hidden_16, train_dataloader, val_dataloader, 200)
accuracy(mlp_1hidden_16, dataloader)

Progress: 100%|███████████████████████████| Time: 0:00:18 (70.34 μs/it)
  epoch:    200
  step:     1329
  loss:     0.09982794936045974
  val_acc:  0.9541333333333334


0.94833

It's still going up, which is a good sign. However, we can see that the training time is also going up a bit, but still nothing too bad on this scale. Now, let's try adding one more hidden layer.

In [14]:
mlp_2hidden = Chain(
    Dense(3 => 32, relu),
    Dense(32 => 16, relu),
    Dense(16 => 1, sigmoid)
) |> f64

train!(mlp_2hidden, train_dataloader, val_dataloader, 200)
accuracy(mlp_2hidden, dataloader)

Progress: 100%|███████████████████████████| Time: 0:00:25 (94.63 μs/it)
  epoch:    200
  step:     1329
  loss:     0.021749399339997
  val_acc:  0.9920666666666667


0.99073

With this architecture, we achieve 99% accuracy. But the training time is now almost double compared to the initial base architecture, even though it's still fast in absolute terms. There's one thing that we can still do to reduce training time whilst keeping the model architecture the same, which going down to Float32 precision. As this is not a drastic reduction in precision in this case, we should keep most/all the predictive performance and decreasing both compute time and memory usage.

In [7]:
# Casting x,y,z values to Float32
coords_f32 = coords |> f32

# Creating new dataloaders
dataloader_f32 = Flux.DataLoader((coords_f32, labels), batchsize=64, shuffle=false)
train_dataloader_f32 = Flux.DataLoader((coords_f32[:,begin:85_000], labels[:,begin:85_000]), batchsize=64, shuffle=true)
val_dataloader_f32 = Flux.DataLoader((coords_f32[:,85_001:end], labels[:,85_001:end]), batchsize=64, shuffle=false)

# New model without calling f64
mlp_2hidden_f32 = Chain(
    Dense(3 => 32, relu),
    Dense(32 => 16, relu),
    Dense(16 => 1, sigmoid)
)

train!(mlp_2hidden_f32, train_dataloader_f32, val_dataloader_f32, 200)
jldsave("mlp_2hidden_f32.jld2", model_state=Flux.state(mlp_2hidden_f32))
accuracy(mlp_2hidden_f32, dataloader_f32)

Progress: 100%|███████████████████████████| Time: 0:00:23 (87.51 μs/it)
  epoch:    200
  step:     1329
  loss:     0.027291745
  val_acc:  0.9879333333333333


0.98927

As we can see, the overall accuracy remained almost unchanged and the execution time per batch decreased 7.5%, from 94.64μs to 87.51μs. This translates to only a 2 second difference, because it's a lightweight model and there's not much data per batch (would be very different for image input, for example), but in a real world scenatio with bigger models or in an embedded setting, this could have a big impact not only in training time but also inference time.

At this point, we have a model with 99% accuracy over the entire dataset. Given this and the goal of this task, we've reached a point of diminishing returns, on which further architecture changes would result in very little improvement. But imagining the results were not satisfactory, we could still dive deeper into bigger architectures and also hyperparameter search, by changing learning rate or adding normalization layers, for example.

## Results analysis and metrics

Now that we have a trained model, we can analyse the predictions with further detail, to have some insights into where the classification is failing. The first thing we should do is know the class distribution on the dataset. We can know that by comparing the number of _signal_ vs _background_ samples.

In [7]:
labels |> sum

22445

So, this tells us there's a significant class imbalance in the dataset. Of the total 100000 samples, 22445 are _signal_ and the remaining 77555 are _background_. This is extremely relevant because accuracy is sensitive to class imbalance. This means that a classifier could learn to classify everything as background and would automatically achieve an accuracy of 77.555%! Does this number sound familiar? This is precisely what happened with the MLP architecture with no hidden layers! The seemingly reasonable accuracy of 0.77555 for a model that size is rooted on this data imbalance. We can confirm this by doing inference over the dataset and also with some random data and checking how many of those prediction would be _signal_.

In [58]:
(mlp_0hidden(coords) .> 0.5) |> sum

0

In [56]:
(mlp_0hidden([rand(100_000) rand(100_000) rand(100_000)]') .> 0.5) |> sum

0

Like in the evaluation, we are set the threshold at 0.5 to distinguish between _signal_ and _background_ and indeed the basic model predicts everything as _background_.

To deal with class imbalance, we could adjust at least the validation dataset to equal representation of both classes or look into metrics that are not sensible to this imbalance. Such metrics would be AUC-ROC for example, which on top of being class imbalance invariant (as it compares TP rate vs FP rate), it's also classification threshold invariant, meaning that the ROC curve is calculated at many threshold values, rather than just our predefined 0.5 value.

As this is out of the scope for this task, we are just going to calculate some more immediate metrics and take a look at those.

**Let's also take this chance to test the model saving/loading.**

In [34]:
# Instantiate model
model = Chain(
    Dense(3 => 32, relu),
    Dense(32 => 16, relu),
    Dense(16 => 1, sigmoid)
)

# Load weights from saved file
Flux.loadmodel!(model, JLD2.load("mlp_2hidden_f32.jld2", "model_state"))

Chain(
  Dense(3 => 32, relu),                 # 128 parameters
  Dense(32 => 16, relu),                # 528 parameters
  Dense(16 => 1, σ),                    # 17 parameters
)                   # Total: 6 arrays, 673 parameters, 3.004 KiB.

In [35]:
accuracy(model, dataloader_f32)

0.98927

Now that we loaded the model and validated the weights are loaded by doing a new evaluation, we can use the following function to calculate the confusion matrix.

In [123]:

"""
Calculate confusion matrix.

# Example

```jldoctest
julia> BitMatrix([[1 1 0 0 1]; [1 0 1 0 1]]) |> confusionmatrix
2×2 Matrix{Int64}:
 2  1
 1  1
```

# Arguments
- `ŷy::BitMatrix`: 2xN matrix containing predictions (row 1) and ground-truths (row 2).
"""
function confusionmatrix(ŷy::BitMatrix)
    fp = tp = tn = fn = 0

    for (ŷ, y) in eachcol(ŷy)
        if ŷ
            ŷ==y ? tp += 1 : fp += 1
        else
            ŷ==y ? tn += 1 : fn += 1
        end
    end

    return [tp fp; fn tn]
end;

The confusion matrix is composed of four different values:
- **True Positives**: _Signal_ correctly classified as _signal_
- **False Positives**: _Signal_ incorrectly classified as _background_
- **False Negatives**: _Background_ incorrectly classified as _signal_
- **True Negatives**: _Background_ correctly classified as _background_

In [111]:
tp, fn, fp, tn = reduce(hcat, [[model(x) .> 0.5; y] for (x,y) in dataloader_f32]) |> confusionmatrix

2×2 Matrix{Int64}:
 22083    711
   362  76844

Now that we have the confusion matrix, we now know that of the misclassified samples, 711 were false positives (_background_ classified as _signal_) and 362 were false positives (_signal_ classified as _background_). One can also analyse different threshold values in an attempt change this distribution, for example if one wants to minimize false positives over false negatives, for example.

We can also use these numbers to calculate more relevant metrics, for example:
- **Accuracy**: As previously seen, it's the number of correct preditions over all predictions.
- **Precision**: Proportion of correctly predicted positive samples among all samples predicted as positive.
- **Recall**: Proportion of correctly predicted positive samples among all actual positive samples.
- **F1-Score**: Harmonic mean of precision and recall, which is a good metric to use on imbalanced datasets such as this.
- **True Negative Rate**: Proportion of correctly predicted negative samples among all actual negative samples
- **False Positive Rate**: Proportion of incorrectly predicted positive samples among all actual negative samples

In [122]:
@show acc = (tp + tn) /(tp + tn + fp + fn)

@show precision = tp / (tp + fp) # Also referred to as Positive Predictive Value
@show recall = tp / (tp + fn) # Also referred to as Recall or Sensitivity
@show f1score = 2 * (precision * recall) / (precision + recall)

@show tnr = tn / (tn + fp) # True Negative Rate, also referred to as Specificity
@show fpr = fp / (fp + tn); # False Positive Rate

accuracy = (tp + tn) / (tp + tn + fp + fn) = 0.98927
precision = tp / (tp + fp) = 0.9688075809423532
recall = tp / (tp + fn) = 0.9838716863443974
f1score = (2 * (precision * recall)) / (precision + recall) = 0.9762815270010389
tnr = tn / (tn + fp) = 0.9908323125523821
fpr = fp / (fp + tn) = 0.00916768744761782



As a next step, we could add the Plots.jl package and plot the Precision-Recall and ROC curves using different threshold values and calculate the areas under those curves to get scalar metrics. This could be something to implement in Julia on a next step, as it provides more information about the model.